In [ ]:
import pandas as pd

### 예제 문제 
1. csv 폴더 안에 있는 tran_1, tran_2 파일 로드 
2. 두개의 데이터프레임을 단순하게 행을 추가하는 결합
3. csv 폴더 안에 있는 tran_d_1, tran_d_2 파일 로드 
4. 두개의 데이터프레임을 단순하게 행을 추가하는 결합 
5. 2번 과정에서 나온 데이터프레임과 4번 과정에서 나온 데이터프레임을 특정한 조건에 맞춰서 열을 결합 기준은 2번 데이터프레임을 기준으로 결합

In [ ]:
# tran_1, tran_2 로드해서 유니언 결합
tran_1 = pd.read_csv("../../csv/tran_1.csv")
tran_2 = pd.read_csv("../../csv/tran_2.csv")
print(tran_1.head(1))
print(tran_2.head(1))

In [ ]:
tran = pd.concat([tran_1, tran_2], axis=0, ignore_index=True)

In [ ]:
tran_d_1 = pd.read_csv('../../csv/tran_d_1.csv')
tran_d_2 = pd.read_csv('../../csv/tran_d_2.csv')
print(tran_d_1.head(1))
print(tran_d_2.head(1))

In [ ]:
tran_d = pd.concat([tran_d_1, tran_d_2], axis=0, ignore_index=True)

In [ ]:
print(tran.head(1))
print(tran_d.head(1))

In [ ]:
# tran, tran_d 두개의 데이터프레임을 조인결합 
tran = pd.merge(tran, tran_d, on = 'transaction_id', how = 'inner')

In [ ]:
tran_d['transaction_id'].value_counts()

1. csv폴더에서 customer_master, item_master 파일 로드 
2. 4개의 데이터프레임이 결합되어 있는 tran 데이터프레임에 조인 결합

In [ ]:
customer = pd.read_csv("../../csv/customer_master.csv")
customer.head(1)

In [ ]:
tran.head(1)

In [ ]:
tran = pd.merge(tran, customer, on='customer_id', how='left')

In [ ]:
item = pd.read_csv('../../csv/item_master.csv')
item.head(1)

In [ ]:
tran.head(1)

In [ ]:
df = pd.merge(tran, item, on='item_id', how='left')

In [ ]:
df.info()

In [ ]:
df[['price', 'quantity', 'item_price']]

1. 거래금액이 가장 큰 데이터는 무엇인가?
2. 거래가 가장 많이 된 물품의 상위 1개를 출력

In [ ]:
df['price'].max()

In [ ]:
# price를 기준으로 내림차순 -> 상위 5개 
df.sort_values('price', ascending=False).head(5)

In [ ]:
## 거래가 가장 많이 된 물품을 확인 
# 1. 아이템을 기준으로 그룹화 -> quantity의 합계
group_data = df[['item_id', 'quantity']].groupby('item_id').sum()
# group_data에서 quantity를 기준으로 내림차순 정렬 -> 상위 3개
group_data.sort_values('quantity', ascending=False).head(3)

In [ ]:
df2 = df.drop('price', axis=1)

1. df2에서 total_price를 생성
2. total_price는 tracsaction_id를 기준으로 거래 금액의 합계
3. price -> item_price * quantity 파생변수 생성
4. transaction_id를 기준으로 그룹화 price의 합계
5. 그룹화 데이터의 컬럼의 이름을 total_price 변경
6. 그룹화한 데이터의 price를 df2 조인결합을 이용해서 열 결합

In [ ]:
df2['price'] = df2['item_price'] * df2['quantity']

In [ ]:
# df2[['transaction_id', 'price']].groupby('transaction_id').sum()
group_data = df2.groupby('transaction_id').sum()[['price']]

In [ ]:
# group_data의 컬럼의 이름을 total_price 변경
group_data.columns = ['total_price']
group_data.head(1)

In [ ]:
group_data.reset_index(inplace=True)

In [ ]:
df2 = pd.merge(df2, 
         group_data, 
         on = 'transaction_id',
         how='left')

In [ ]:
df.head(10)

In [ ]:
df2.head(10)

In [ ]:
(df['price'] == df2['total_price']).value_counts()

In [62]:
df.tail(5)

,transaction_id,price,payment_date,customer_id,detail_id,item_id,quantity,name,class,gender,start_date,end_date,campaign_id,is_deleted,item_name,item_price
7139,T0000006894,180000,2019-07-31 21:20:44,HI400734,7139,S004,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PC-D,180000
7140,T0000006895,85000,2019-07-31 21:52:48,AS339451,7140,S002,1,XXXXX,C01,M,2016-07-01 00:00:00,NaN,CA2,0.0,PC-B,85000
7141,T0000006896,100000,2019-07-31 23:35:25,OA027325,7141,S001,2,XX,C03,F,2018-12-02 00:00:00,NaN,CA3,0.0,PC-A,50000
7142,T0000006897,85000,2019-07-31 23:39:35,TS624738,7142,S002,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PC-B,85000
7143,T0000006898,85000,2019-07-31 23:41:38,AS834214,7143,S002,1,XXXXX,C01,M,2017-05-01 00:00:00,2018-05-31 00:00:00,CA1,1.0,PC-B,85000


1. payment_date컬럼의 데이터의 타입을 시계열로 변경
2. gender컬럼의 데이터를 M -> 남자, F -> 여자 로 데이터를 변경
3. 시계열에서 데이터에서 년도-월 따로 추출해서 payment_month 생성

In [64]:
# 시계열로 데이터의 타입을 변경 
df['payment_date'] = \
     pd.to_datetime(df['payment_date'], format= "%Y-%m-%d %H:%M:%S")

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7144 entries, 0 to 7143
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   transaction_id  7144 non-null   object        
 1   price           7144 non-null   int64         
 2   payment_date    7144 non-null   datetime64[ns]
 3   customer_id     7144 non-null   object        
 4   detail_id       7144 non-null   int64         
 5   item_id         7144 non-null   object        
 6   quantity        7144 non-null   int64         
 7   name            6069 non-null   object        
 8   class           6069 non-null   object        
 9   gender          6069 non-null   object        
 10  start_date      6069 non-null   object        
 11  end_date        1978 non-null   object        
 12  campaign_id     6069 non-null   object        
 13  is_deleted      6069 non-null   float64       
 14  item_name       7144 non-null   object        
 15  item